In [3]:
import pandas as pd
import dask.dataframe as dd
dataloc = '../data/'

In [4]:
games_1 = dd.read_csv(dataloc + 'raw/csv/Games_1.csv')
'''
games_genres = pd.read_csv(dataloc + 'raw/csv/Games_Genres.csv')
games_info = pd.read_csv(dataloc + 'raw/csv/app_id_info_2.csv')
games_1_genres = pd.read_csv(dataloc + 'interim/export/Games_1_Genres_800k.csv').drop(columns=['dateretrieved'])
steamspy = pd.read_json(dataloc + 'external/steamspy.json', orient='index').sort_values(by=['appid'])
games_processed = pd.read_csv(dataloc + 'interim/export/games_info_ratings.csv')
'''

"\ngames_genres = pd.read_csv(dataloc + 'raw/csv/Games_Genres.csv')\ngames_info = pd.read_csv(dataloc + 'raw/csv/app_id_info_2.csv')\ngames_1_genres = pd.read_csv(dataloc + 'interim/export/Games_1_Genres_800k.csv').drop(columns=['dateretrieved'])\nsteamspy = pd.read_json(dataloc + 'external/steamspy.json', orient='index').sort_values(by=['appid'])\ngames_processed = pd.read_csv(dataloc + 'interim/export/games_info_ratings.csv')\n"

In [10]:
final_users = dd.read_csv(dataloc + 'processed/user_info.csv')

# Find games the final users have played to test against

In [11]:
set_of_users = final_users.drop_duplicates('steamid')[:1000]

TypeError: '<' not supported between instances of 'str' and 'int'

In [6]:
final_games = final_users.merge(games_1, on='steamid', how='inner', suffixes=('', '_y'))

In [14]:
final_reparted = final_games.sample(frac=0.20).repartition(npartitions=1).compute()
final_reparted = final_reparted.dropna()
final_reparted.to_csv(dataloc + 'processed/all_user_games.csv', index=False)

# Clean steamspy data

In [45]:
droppedspy = steamspy.drop(columns=['discount', 'initialprice', 'owners', 'price', 'score_rank', 'userscore', 'name'])
appinfo = games_processed[['appid', 'Genre', 'Title', 'Rating']].drop_duplicates(subset='appid', keep='first')
appinfo.head()

gamespy_merge = droppedspy.merge(appinfo, on='appid', how='inner', suffixes=('', '_Y'))
gamespy_merge.to_csv(dataloc + 'processed/export/games_info.csv', index=False)

# Extra cleaning of user data

In [49]:
games_processed.head()
#games_processed.drop(columns=['Genre', 'Title', 'Rating'], inplace=True)
games_processed.to_csv(dataloc + 'processed/export/user_info.csv', index=False)

# Game info with rating
Clean game info table

##### Remove Genre column after realising it isn't as one column

In [4]:
temp = pd.read_csv(dataloc + 'processed/export/games_info.csv').drop('Genre', axis=1)
temp.to_csv(dataloc + 'processed/export/games_info.csv', index=False)

##### Also join game genre with game info

In [8]:
genre = pd.read_csv(dataloc + 'processed/export/games_genres_ohe.csv')
new_games_info = temp.merge(genre, on='appid', how='inner', suffixes=('', '_y'))
new_games_info.to_csv(dataloc + 'processed/export/games_info_v2.csv', index=False)

### Keeping on

In [3]:
games_info_with_rating = games_info[games_info.Rating > -1]
games_info_with_rating = games_info_with_rating[games_info_with_rating.Type != 'demo']
games_info_with_rating = games_info_with_rating[games_info_with_rating.Type != 'video']
games_info_with_rating.drop(columns=['Type', 'Price', 'Release_Date', 'Required_Age', 'Is_Multiplayer'], inplace=True)
games_info_with_rating.Type.nunique()

AttributeError: 'DataFrame' object has no attribute 'Type'

# Merge games info with games_1

In [6]:
games_info_with_rating.head()

,appid,Title,Rating
26,4780,Medieval II: Total War™ Kingdoms,85
31,65980,Sid Meier's Civilization®: Beyond Earth™,81
67,25980,Majesty 2,72
73,207420,Wizorb,81
86,233210,Air Conflicts: Vietnam,43


In [4]:
merged_info_games = games_1_genres.merge(games_info_with_rating, on='appid', how='inner', suffixes=('', '_y'))
merged_info_games.rename(columns={'playtime_2weeks_sum': 'user_2weeks_playtime'}, inplace=True)

In [5]:
merged_info_games.head()

,steamid,appid,playtime_2weeks,playtime_forever,user_2weeks_playtime,Genre,Title,Rating
0,76561198014073590,55230,143.0,392.0,932.0,Action,Saints Row: The Third,84
1,76561198012546870,55230,686.0,1876.0,1314.0,Action,Saints Row: The Third,84
2,76561198012666560,55230,741.0,741.0,2141.0,Action,Saints Row: The Third,84
3,76561198012327490,55230,82.0,82.0,82.0,Action,Saints Row: The Third,84
4,76561198014451410,55230,30.0,2067.0,689.0,Action,Saints Row: The Third,84


In [8]:
merged_info_games.sort_values('steamid', ascending=False).to_csv(dataloc + 'processed/export/games_info_ratings.csv', index=False)

# Get most played genre per user

######  TODO maybe

# Users weekly cleaning

In [ ]:
users_weekly = pd.read_csv(dataloc + 'interim/export/User_Weekly_Time.csv')

In [ ]:
users_weekly['playtime_2weeks'].max()

In [ ]:
filtered_games = games_1[games_1['playtime_2weeks'] > 0.0]

In [7]:
games_1.head()

,steamid,appid,playtime_2weeks,playtime_forever,dateretrieved
0,76561198015117560,41500,NaN,1785.0,2013-06-25 08:26:33
1,76561198015117560,400,NaN,5.0,2013-06-25 08:26:33
2,76561198015117470,8600,NaN,5.0,2013-06-25 08:26:22
3,76561198015117470,8660,NaN,1.0,2013-06-25 08:26:22
4,76561198015117430,41500,NaN,1708.0,2013-06-25 08:26:18


# Genre cleaning

In [9]:
genres_of_interest = ['Action', 'Free to Play', 'Strategy', 'Adventure', 'Indie', 'RPG', 'Casual', 'Simulation', 'Racing', 'Massively Multiplayer', 'Sports', 'Early Access']

In [10]:
games_genres['Genre'].unique()

array(['Action', 'Free to Play', 'Strategy', 'Adventure', 'Indie', 'RPG',
       'Animation & Modeling', 'Video Production', 'Casual', 'Simulation',
       'Racing', 'Massively Multiplayer', 'Sports', 'Early Access',
       'Photo Editing', 'Utilities', 'Design & Illustration', 'Education',
       'Software Training', 'Web Publishing', 'Audio Production',
       'Accounting'], dtype=object)

In [ ]:
agged_games = pd.read_csv(dataloc + 'processed/export/Games_1_Genres_800k-3.csv')

In [ ]:
print(agged_games['Genre'].nunique())
print(len(agged_games.index))

In [ ]:
merged_agged.head()

# Filter wrong time data from users weekly
Maximum amount of minutes possible in two weeks is 20160

In [ ]:
users_weekly = users_weekly[users_weekly['playtime_2weeks'] < 20160]

In [ ]:
users_weekly.rename(columns={'playtime_2weeks': 'playtime_2weeks_sum'}, inplace=True)

In [ ]:
users_weekly.head(5)

In [ ]:
users_weekly.to_csv(dataLoc + 'processed/Users_Weekly.csv', index=False)

# Join datasets

In [ ]:
joined = games_1.merge(users_weekly, on='steamid', how='inner').dropna()

In [ ]:
joined_genres = joined.merge(games_genres, on='appid', how='inner')
joined_genres.set_index('steamid')

#### Sort dataframe on steamid

In [ ]:
sorted_genres = joined_genres.set_index('steamid')

In [ ]:
joined_genres.head(5)

In [ ]:
joined['steamid'].nunique().compute()

# Repartition the smaller dataset into 1 partition

In [ ]:
small = joined_genres.sample(frac=0.0475)
reparted_small = small.repartition(npartitions=1).compute()

In [ ]:
print(len(small.index))

In [ ]:
sorted_small = reparted_small.set_index('steamid')

In [ ]:
reparted_small.to_csv(dataLoc + 'processed/export/Games_1_Genres_800k-3.csv', index=False)

# Get total playtime over 2 weeks and most played genre per user
Group by steam id, genre and sum by playtime 2weeks

In [ ]:
users_weekly = games_1.groupby(['steamid'])['playtime_2weeks'].sum().reset_index().compute()

In [ ]:
users_weekly.head()

In [ ]:
top_genre = games_1.compute().sort_values('playtime_2weeks', ascending=False).drop_duplicates(['steamid'])

In [ ]:
top_genre.head()

In [ ]:
users_top_weekly = games_1.groupby('steamid')['appid']

In [ ]:
users_top_weekly.head()

# TODO
Get game with max playtime_2weeks per user

<h1>Filter out zero playtime</h1>

In [ ]:
users2 = users_weekly[users_weekly['playtime_2weeks'] > 0.0]

In [ ]:
users2.head(5)

In [ ]:
users2['playtime_2weeks'].max()

In [ ]:
users2.to_csv(dataLoc + 'interim/export/User_Weekly_Time.csv', index=False)

In [ ]:
len(users2.index)